In [1]:
print("Hello World!")

Hello World!


In [3]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all FutureWarning warnings that might flood the console log
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os
import pandas as pd


import time
from utils import create_patches_v3, create_binary_mask, create_patches_v2, create_patches_v1, create_patches_coordinates, create_patches_coordinates_v1
from scipy.stats.contingency import crosstab
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

# Loading directory
wsi_dir = "/root/PriCE/dataset/1WSI/data/"
patch_size = 224
cpu_core = 12

# Saving directory
save_dir = wsi_dir

downsize = patch_size
patch_extraction_size = patch_size
mask_overlap = 80.0
batch_size = 64
cpu_workers = cpu_core
use_prob_threshold = 0.8 # None  # whether to give final prediction {0,1} based on certain probability

# torch.manual_seed(250)
print('--------------Start--------------')
# read the files
wsi_files = os.listdir(wsi_dir)
# get all files except temp directory containing patches
wsi_files = [f for f in wsi_files if f.endswith("svs") or f.endswith("ndpi") or f.endswith("mrxs")]

print(f"Total files in {wsi_dir} directory are {len(wsi_files)}")

path = os.path.join(wsi_dir, "cnn_ensemble_updated")
if not os.path.exists(path):
    os.mkdir(path)

lst_patching_time = []

stat_NMI_graph = [] 
stat_NMI_base = []
coords = []
new_file_paths = []
# start patching process over a certain WSI
for i, f in enumerate(wsi_files):

    st_binarymask = time.time()
    new_file_path = os.path.join(path, f.split(".")[0])
    print(new_file_path)

    # just take the name not extension
    if not os.path.exists(new_file_path):
        os.mkdir(new_file_path)
        
    # find binary mask to locate tissue on WSI            
    w, h = create_binary_mask(wsi_dir, f, new_file_path, downsize=patch_size)

    # print(w/224, h/224)
    et_binarymask = time.time()

    binarymask_processtime = et_binarymask - st_binarymask
    print(f"Time spent on create_binary_mask is {binarymask_processtime} for {f}")
        
    st_patching = time.time()
    patch_folder = os.path.join(new_file_path, "patches")
    
    if not os.path.exists(patch_folder):
    # # else:
        os.mkdir(patch_folder) 
    if os.path.exists(patch_folder):
    #     os.rmdir(patch_folder)
    # elif not os.path.exists(patch_folder):
    # # else:
    #     os.mkdir(patch_folder) 
        # coordinates, id_to_coordinates = create_patches_coordinates(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        # coordinates, id_to_coordinates, patches = create_patches_v1(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        
        # create_patches_v3(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # 1. create coordinates with multi-processing
        res = create_patches_coordinates_v1(wsi_dir, f, new_file_path, workers=cpu_workers, patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # patches = create_patches_v2(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
            # patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        # print(len(coordinates))
        # print(len(patches))
        # print(id_to_coordinates)

        # print("Coordinates Done!", len(res), res[:3])
        # coordinates = list(res.values())
        coordinates = res.values()
        ids = res.keys()
        # print("Coordinates Done!", len(res), len(coordinates), coordinates[:3])
        coords.append(coordinates)
        new_file_paths.append(new_file_path)
        

--------------Start--------------
Total files in /root/PriCE-1/dataset/1WSI/data/ directory are 1
/root/PriCE-1/dataset/1WSI/data/cnn_ensemble_updated/TCGA-E9-A1N3-01Z-00-DX1

##########################################
Creating basic binary masks for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs
Time taken for creating binary mask 0.02 seconds
Time spent on create_binary_mask is 0.018831253051757812 for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs
Create patches for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs, using 12 CPUs out of 12
Patches coordinates created for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs in 0.19 minutes.


In [6]:
new_file_paths = ['/root/PriCE/dataset/1WSI/data/cnn_ensemble_updated/TCGA-E9-A1N3-01Z-00-DX1']

In [3]:
import os
import fnmatch
def count_files_with_numeric_pattern(start_path, file_pattern):
    count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                count += 1
                # print(os.path.join(root, file))  # 打印匹配的文件路径
    return count

In [7]:
print(new_file_paths)

['/root/PriCE/dataset/1WSI/data/cnn_ensemble_updated/TCGA-E9-A1N3-01Z-00-DX1']


# Simulation Data Collection

## Graph Splitting based simulation

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data

# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    # strategies = ['saturation_largest_first']
    strategies = ['saturation_largest_first', 'smallest_last', 'connected_sequential', 'independent_set', 'largest_first', 'random_sequential']

    time_per_strategy = {}
    for my_strategy in strategies: 
        
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        exe_tmp = []
        datasetSize = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            exetime4data_no = data_no * time_base
            # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
            exe_tmp.append(exetime4data_no)
            datasetSize.append(data_no)
        print(my_strategy, exe_tmp)
        # time_per_strategy[my_strategy] = tmp
        # print('Time_per_strategy: ', time_per_strategy)
        # time = time_per_strategy['saturation_largest_first']
        # exe_time = exe_tmp

        # lantency = No.of patches x 1/throughput_T4
        throughput_T4 = 200   
        time = exe_tmp
        print(datasetSize)
        # unit price
        UP = {"Tesla T4 (gpu-e)": 0,
            # "Tesla T4 (gpu-f)": 0,
            # "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti NO": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }


        # location 
        location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
            # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
            # "RTX 3090 (TY)": "Valencia, Spain",
            "RTX 3090 (UiS)": "Stavanger, Norway",
            "A100 80GB": "Illinois, USA", 
            "A100 40GB": "Illinois, USA", 
            "V100 IL": "Illinois, USA",
            "RTX A6000 IL": "Illinois, USA",
            "RTX A6000 NY": "New York, USA",
            "RTX A5000 IL": "Illinois, USA",
            "RTX A5000 NV": "Nevada, USA",
            "RTX A5000 NY": "New York, USA",
            "RTX A4000 NY": "New York, USA",
            "RTX A4000 IL": "Illinois, USA",
            "A40 IL": "Illinois, USA", 
            "A40 NY": "New York, USA", 
            "A40 NV": "Nevada, USA", 
            "QUADRO RTX 5000 IL": "Illinois, USA", 
            "QUADRO RTX 5000 NY": "New York, USA", 
            "QUADRO RTX 5000 NV": "Nevada, USA", 
            "QUADRO RTX 4000 IL": "Illinois, USA",
            "QUADRO RTX 4000 NY": "New York, USA",
            "QUADRO RTX 4000 NV": "Nevada, USA",
            "RTX 3060 Ti NO": "Stavanger, Norway",
            "RTX 3090 NO": "Stavanger, Norway",
            "RTX 3090 IS": "Iceland",
            "RTX 3080 NO": "Stavanger, Norway",
            "RTX 3080 IS": "Iceland",
            "V100 IN": "India"
            }

        # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
        SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                    "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                    # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                    "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                    "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                    "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                    "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                    "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                    }

        # print(trans_time)
        # lantency = No.of patches x 1/throughput_T4
        # throughput_T4 = 200   
        # time = []

        time = [n/throughput_T4 for n in datasetSize] 
        # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
        # transmission  = No.of patches * 100KB / speed(upload)    # Let 
        trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
            "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
            }


        # compute time
        T3090 = [round(i/2.23, 2) for i in time]
        compt_d = {"Tesla T4 (gpu-e)": time,
            # "Tesla T4 (gpu-f)": time,
            # "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        # tot_time = compt_time+trans_time
        computeDf = pd.DataFrame(data=compt_d)
        transDF = pd.DataFrame(data=trans_time)

        tottDf = computeDf+transDF
        costDf = tottDf * UP
        print(tottDf)

        print(costDf)
        matrix = tottDf.to_numpy()
        cmatrix = costDf.to_numpy()

saturation_largest_first [25.490000000000002, 25.505, 25.505, 25.51]
[5098, 5101, 5101, 5102]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  \
0        126.791540       54.387657  201.266355  201.266355  202.066355   
1        126.866153       54.422937  201.384365  201.384365  202.184365   
2        126.866153       54.422937  201.384365  201.384365  202.184365   
3        126.891023       54.431363  201.420369  201.420369  202.220369   

   RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  ...  \
0    193.836355    148.260230    197.486355    145.289436    151.910230  ...   
1    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
2    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
3    193.990369    148.378484    197.650369    145.412496    152.038484  ...   

   QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  \
0          151.699436          210.746355          165.170230   

In [9]:
print(UP)
print(len(UP.keys()))

{'Tesla T4 (gpu-e)': 0, 'RTX 3090 (UiS)': 0, 'A100 80GB': 2.15, 'A100 40GB': 2.01, 'V100 IL': 0.81, 'RTX A6000 IL': 1.27, 'RTX A6000 NY': 1.27, 'RTX A5000 IL': 0.78, 'RTX A5000 NV': 0.78, 'RTX A5000 NY': 0.78, 'RTX A4000 NY': 0.63, 'RTX A4000 IL': 0.63, 'A40 IL': 1.27, 'A40 NY': 1.27, 'A40 NV': 1.27, 'QUADRO RTX 5000 IL': 0.59, 'QUADRO RTX 5000 NY': 0.59, 'QUADRO RTX 5000 NV': 0.59, 'QUADRO RTX 4000 IL': 0.28, 'QUADRO RTX 4000 NY': 0.28, 'QUADRO RTX 4000 NV': 0.28, 'RTX 3060 Ti NO': 0.34, 'RTX 3090 NO': 0.59, 'RTX 3090 IS': 0.59, 'RTX 3080 NO': 0.5, 'RTX 3080 IS': 0.5, 'V100 IN': 1.07}
27


## Evenly Splitting based Simulation

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data

# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    dataset_numbers = [4,5,6,7,8]
    strategies = ['is_shuffled_True', 'is_shuffled_False']
    time_per_strategy = {}

    for my_strategy in strategies: 
       # 设置你的目标文件夹名称和文件模式
        for dataset_number in dataset_numbers:  
            even_split_file = f'{new_file_path}/even_split/{dataset_number}/'
            # file_pattern = f"*[0-9]{my_strategy}.csv"

            # 计算文件数量
            # dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

            exe_tmp = []
            datasetSize = []
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{even_split_file}even_patchesnames_{str(j)}_{my_strategy}.csv'
                df = pd.read_csv(origfpath)
                data_no = len(df['PatchName'])
                exetime4data_no = data_no * time_base
                # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
                exe_tmp.append(exetime4data_no)
                datasetSize.append(data_no)
            print(my_strategy, dataset_number, exe_tmp)
            # time_per_strategy[my_strategy] = tmp
            # print('Time_per_strategy: ', time_per_strategy)
            # time = time_per_strategy['saturation_largest_first']
            # exe_time = exe_tmp

            # lantency = No.of patches x 1/throughput_T4
            throughput_T4 = 200   
            time = exe_tmp
            print(datasetSize)
            # unit price
            UP = {"Tesla T4 (gpu-e)": 0,
                # "Tesla T4 (gpu-f)": 0,
                # "RTX 3090 (TY)": 0,
                "RTX 3090 (UiS)": 0,
                "A100 80GB": 2.15, 
                "A100 40GB": 2.01, 
                "V100 IL": 0.81,
                "RTX A6000 IL": 1.27,
                "RTX A6000 NY": 1.27,
                "RTX A5000 IL": 0.78,
                "RTX A5000 NV": 0.78,
                "RTX A5000 NY": 0.78,
                "RTX A4000 NY": 0.63,
                "RTX A4000 IL": 0.63,
                "A40 IL": 1.27, 
                "A40 NY": 1.27, 
                "A40 NV": 1.27, 
                "QUADRO RTX 5000 IL": 0.59, 
                "QUADRO RTX 5000 NY": 0.59, 
                "QUADRO RTX 5000 NV": 0.59, 
                "QUADRO RTX 4000 IL": 0.28,
                "QUADRO RTX 4000 NY": 0.28,
                "QUADRO RTX 4000 NV": 0.28,
                "RTX 3060 Ti NO": 0.34,
                "RTX 3090 NO": 0.59,
                "RTX 3090 IS": 0.59,
                "RTX 3080 NO": 0.50,
                "RTX 3080 IS": 0.50,
                "V100 IN": 1.07
                }


            # location 
            location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
                # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
                # "RTX 3090 (TY)": "Valencia, Spain",
                "RTX 3090 (UiS)": "Stavanger, Norway",
                "A100 80GB": "Illinois, USA", 
                "A100 40GB": "Illinois, USA", 
                "V100 IL": "Illinois, USA",
                "RTX A6000 IL": "Illinois, USA",
                "RTX A6000 NY": "New York, USA",
                "RTX A5000 IL": "Illinois, USA",
                "RTX A5000 NV": "Nevada, USA",
                "RTX A5000 NY": "New York, USA",
                "RTX A4000 NY": "New York, USA",
                "RTX A4000 IL": "Illinois, USA",
                "A40 IL": "Illinois, USA", 
                "A40 NY": "New York, USA", 
                "A40 NV": "Nevada, USA", 
                "QUADRO RTX 5000 IL": "Illinois, USA", 
                "QUADRO RTX 5000 NY": "New York, USA", 
                "QUADRO RTX 5000 NV": "Nevada, USA", 
                "QUADRO RTX 4000 IL": "Illinois, USA",
                "QUADRO RTX 4000 NY": "New York, USA",
                "QUADRO RTX 4000 NV": "Nevada, USA",
                "RTX 3060 Ti NO": "Stavanger, Norway",
                "RTX 3090 NO": "Stavanger, Norway",
                "RTX 3090 IS": "Iceland",
                "RTX 3080 NO": "Stavanger, Norway",
                "RTX 3080 IS": "Iceland",
                "V100 IN": "India"
                }

            # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
            SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                        "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                        # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                        "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                        "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                        "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                        "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                        "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                        }

            # print(trans_time)
            # lantency = No.of patches x 1/throughput_T4
            # throughput_T4 = 200   
            # time = []

            time = [n/throughput_T4 for n in datasetSize] 
            # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
            # transmission  = No.of patches * 100KB / speed(upload)    # Let 
            trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
                "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
                }


            # compute time
            T3090 = [round(i/2.23, 2) for i in time]
            compt_d = {"Tesla T4 (gpu-e)": time,
                # "Tesla T4 (gpu-f)": time,
                # "RTX 3090 (TY)": T3090,
                "RTX 3090 (UiS)": T3090,
                "A100 80GB": [round(i*1.55, 2) for i in T3090], 
                "A100 40GB": [round(i*1.55, 2) for i in T3090], 
                "V100 IL": [round(i*1.62, 2) for i in T3090],
                "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
                "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
                "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
                "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
                "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
                "A40 IL": [round(i*0.93, 2) for i in T3090], 
                "A40 NY": [round(i*0.93, 2) for i in T3090], 
                "A40 NV": [round(i*0.93, 2) for i in T3090], 
                "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
                "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
                "RTX 3090 NO": T3090,
                "RTX 3090 IS": T3090,
                "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
                "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
                "V100 IN": [round(i*1.62, 2) for i in T3090]
                }

            # tot_time = compt_time+trans_time
            computeDf = pd.DataFrame(data=compt_d)
            transDF = pd.DataFrame(data=trans_time)

            tottDf = computeDf+transDF
            costDf = tottDf * UP
            print(tottDf)

            print(costDf)
            matrix = tottDf.to_numpy()
            cmatrix = costDf.to_numpy()

is_shuffled_True 4 [25.505, 25.505, 25.5, 25.5]
[5101, 5101, 5100, 5100]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  \
0        126.866153       54.422937  201.384365  201.384365  202.184365   
1        126.866153       54.422937  201.384365  201.384365  202.184365   
2        126.841282       54.404510  201.338362  201.338362  202.138362   
3        126.841282       54.404510  201.338362  201.338362  202.138362   

   RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  ...  \
0    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
1    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
2    193.908362    148.314357    197.558362    145.340966    151.964357  ...   
3    193.908362    148.314357    197.558362    145.340966    151.964357  ...   

   QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  \
0          151.786731          210.884365          165.281421   
1          151.78673

# Minimize the maximum time under the user's budget
* Graph-based splitting strategies

* Budget = 100
* Budget = 120

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl
import pulp


budget = 100

def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    # strategies = ['saturation_largest_first']
    strategies = ['saturation_largest_first', 'smallest_last', 'connected_sequential', 'independent_set', 'largest_first', 'random_sequential']

    time_per_strategy = {}
    solutions4strategies = {}
    for my_strategy in strategies: 
        
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        exe_tmp = []
        datasetSize = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            exetime4data_no = data_no * time_base
            
            exe_tmp.append(exetime4data_no)
            datasetSize.append(data_no)
        print(my_strategy, exe_tmp)
        # time_per_strategy[my_strategy] = tmp
        # print('Time_per_strategy: ', time_per_strategy)
        # time = time_per_strategy['saturation_largest_first']
        # exe_time = exe_tmp

        # lantency = No.of patches x 1/throughput_T4
        throughput_T4 = 200   
        time = exe_tmp
        print(datasetSize)
        # unit price
        UP = {"Tesla T4 (gpu-e)": 0,
            # "Tesla T4 (gpu-f)": 0,
            # "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti NO": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }


        # location 
        location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
            # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
            # "RTX 3090 (TY)": "Valencia, Spain",
            "RTX 3090 (UiS)": "Stavanger, Norway",
            "A100 80GB": "Illinois, USA", 
            "A100 40GB": "Illinois, USA", 
            "V100 IL": "Illinois, USA",
            "RTX A6000 IL": "Illinois, USA",
            "RTX A6000 NY": "New York, USA",
            "RTX A5000 IL": "Illinois, USA",
            "RTX A5000 NV": "Nevada, USA",
            "RTX A5000 NY": "New York, USA",
            "RTX A4000 NY": "New York, USA",
            "RTX A4000 IL": "Illinois, USA",
            "A40 IL": "Illinois, USA", 
            "A40 NY": "New York, USA", 
            "A40 NV": "Nevada, USA", 
            "QUADRO RTX 5000 IL": "Illinois, USA", 
            "QUADRO RTX 5000 NY": "New York, USA", 
            "QUADRO RTX 5000 NV": "Nevada, USA", 
            "QUADRO RTX 4000 IL": "Illinois, USA",
            "QUADRO RTX 4000 NY": "New York, USA",
            "QUADRO RTX 4000 NV": "Nevada, USA",
            "RTX 3060 Ti NO": "Stavanger, Norway",
            "RTX 3090 NO": "Stavanger, Norway",
            "RTX 3090 IS": "Iceland",
            "RTX 3080 NO": "Stavanger, Norway",
            "RTX 3080 IS": "Iceland",
            "V100 IN": "India"
            }

        # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
        SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                    "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                    # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                    "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                    "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                    "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                    "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                    "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                    }

        # print(trans_time)
        # lantency = No.of patches x 1/throughput_T4
        # throughput_T4 = 200   
        # time = []

        time = [n/throughput_T4 for n in datasetSize] 
        # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
        # transmission  = No.of patches * 100KB / speed(upload)    # Let 
        trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
            "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
            }


        # compute time
        T3090 = [round(i/2.23, 2) for i in time]
        compt_d = {"Tesla T4 (gpu-e)": time,
            # "Tesla T4 (gpu-f)": time,
            # "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        # tot_time = compt_time+trans_time
        computeDf = pd.DataFrame(data=compt_d)
        transDF = pd.DataFrame(data=trans_time)

        tottDf = computeDf+transDF
        costDf = tottDf * UP
        # print(df1)
        # matrix = tottDf.to_numpy()
        # cmatrix = costDf.to_numpy()

        
        time_matrix = tottDf.to_numpy()
        cost_matrix = costDf.to_numpy()
        print(costDf)
        

        n, m = time_matrix.shape  # n行m列
        print(n, m)

        # 定义最大总成本ε
        # max_total_cost = 100  # 这个值根据实际情况调整
        # max_total_cost = 120  # 这个值根据实际情况调整
        max_total_cost = budget  # 这个值根据实际情况调整

        # 创建问题实例
        prob = pulp.LpProblem("MinimizeMaxTimeUnderCostConstraint", pulp.LpMinimize)

        # 定义决策变量
        x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')
        # 定义一个辅助变量来表示最大完成时间
        max_time = pulp.LpVariable("max_time", lowBound=0)

        # 目标函数：最小化最大完成时间
        prob += max_time

        # 约束条件
        # 每行恰好选择一个元素
        for i in range(n):
            prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1

        # 每列最多选择一个元素
        for j in range(m):
            prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1

        # 总成本不超过ε
        prob += pulp.lpSum(x[(i, j)] * cost_matrix[i, j] for i in range(n) for j in range(m)) <= max_total_cost

        # 最大完成时间约束
        for i in range(n):
            for j in range(m):
                prob += x[(i, j)] * time_matrix[i, j] <= max_time

        # 求解问题
        prob.solve()

        # 检查是否有最优解
        if pulp.LpStatus[prob.status] == "Optimal":
            solution = [j for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) > 0.5]
            total_cost = sum(cost_matrix[i, solution[i]] for i in range(n))
            max_time_value = pulp.value(max_time)
            assignment = [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
            print(f"Solution: {solution}, Total Cost: {total_cost}, Max Time: {max_time_value}, Assignment: {assignment}")
            solution_per_strategy = {'Total Cost': total_cost,
                                'Max Time': max_time_value,
                                'Solution': solution,
                                'assignment': assignment }
            solutions4strategies[my_strategy] = solution_per_strategy
        else:
            print("No optimal solution found.")
            continue 

        print(my_strategy) 
        
        # # 打印解决方案
        # for solution in solutions:
        #     print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
        # # print(solutions)
   

        

saturation_largest_first [25.490000000000002, 25.505, 25.505, 25.51]
[5098, 5101, 5101, 5102]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  \
0               0.0             0.0  432.722662  404.545373  163.673747   
1               0.0             0.0  432.976386  404.782575  163.769336   
2               0.0             0.0  432.976386  404.782575  163.769336   
3               0.0             0.0  433.053793  404.854942  163.798499   

   RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  ...  \
0    246.172170    188.290492    154.039357    113.325760    118.489979  ...   
1    246.322044    188.406304    154.139205    113.401650    118.568908  ...   
2    246.322044    188.406304    154.139205    113.401650    118.568908  ...   
3    246.367769    188.440675    154.167288    113.421747    118.590018  ...   

   QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  \
0           89.502667           59.008979           46.247664   

In [14]:
print(solutions4strategies)
print(len(solutions4strategies.values()))

{'saturation_largest_first': {'Total Cost': 81.49341761112282, 'Max Time': 62.847657, 'Solution': [21, 1, 22, 24], 'assignment': [(0, 21), (1, 1), (2, 22), (3, 24)]}, 'smallest_last': {'Total Cost': 79.21221733726564, 'Max Time': 60.115429, 'Solution': [24, 1, 22, 21, 0], 'assignment': [(0, 24), (1, 1), (2, 22), (3, 21), (4, 0)]}, 'connected_sequential': {'Total Cost': 86.17776927628933, 'Max Time': 58.671116, 'Solution': [1, 24, 22, 21, 18, 0], 'assignment': [(0, 1), (1, 24), (2, 22), (3, 21), (4, 18), (5, 0)]}, 'independent_set': {'Total Cost': 90.26339986885922, 'Max Time': 57.216804, 'Solution': [24, 22, 1, 21, 18, 8, 25], 'assignment': [(0, 24), (1, 22), (2, 1), (3, 21), (4, 18), (5, 8), (6, 25)]}, 'largest_first': {'Total Cost': 85.667821291027, 'Max Time': 44.986267, 'Solution': [22, 1, 24, 21, 25, 0, 23, 20], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 25), (5, 0), (6, 23), (7, 20)]}, 'random_sequential': {'Total Cost': 97.26354823929992, 'Max Time': 43.993731, 'Solut

# Minimize the makespan under the user's budget
* Evenly Splitting Solutions
* Budget=100

* Budget=120

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import pulp

budget = 100

def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

# Even_solutions4strategies= {}
for new_file_path in new_file_paths: 
    dataset_numbers = [4,5,6,7,8]
    strategies = ['is_shuffled_True', 'is_shuffled_False']
    time_per_strategy = {}
    Even_solutions4strategies = {}
    for my_strategy in strategies: 
       # 设置你的目标文件夹名称和文件模式
        for dataset_number in dataset_numbers:  
            even_split_file = f'{new_file_path}/even_split/{dataset_number}/'
            # file_pattern = f"*[0-9]{my_strategy}.csv"

            # 计算文件数量
            # dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

            exe_tmp = []
            datasetSize = []
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{even_split_file}even_patchesnames_{str(j)}_{my_strategy}.csv'
                df = pd.read_csv(origfpath)
                data_no = len(df['PatchName'])
                exetime4data_no = data_no * time_base
                # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
                exe_tmp.append(exetime4data_no)
                datasetSize.append(data_no)
            print(my_strategy, dataset_number, exe_tmp)
            # time_per_strategy[my_strategy] = tmp
            # print('Time_per_strategy: ', time_per_strategy)
            # time = time_per_strategy['saturation_largest_first']
            # exe_time = exe_tmp

            # lantency = No.of patches x 1/throughput_T4
            throughput_T4 = 200   
            time = exe_tmp
            print(datasetSize)
            # unit price
            UP = {"Tesla T4 (gpu-e)": 0,
                # "Tesla T4 (gpu-f)": 0,
                # "RTX 3090 (TY)": 0,
                "RTX 3090 (UiS)": 0,
                "A100 80GB": 2.15, 
                "A100 40GB": 2.01, 
                "V100 IL": 0.81,
                "RTX A6000 IL": 1.27,
                "RTX A6000 NY": 1.27,
                "RTX A5000 IL": 0.78,
                "RTX A5000 NV": 0.78,
                "RTX A5000 NY": 0.78,
                "RTX A4000 NY": 0.63,
                "RTX A4000 IL": 0.63,
                "A40 IL": 1.27, 
                "A40 NY": 1.27, 
                "A40 NV": 1.27, 
                "QUADRO RTX 5000 IL": 0.59, 
                "QUADRO RTX 5000 NY": 0.59, 
                "QUADRO RTX 5000 NV": 0.59, 
                "QUADRO RTX 4000 IL": 0.28,
                "QUADRO RTX 4000 NY": 0.28,
                "QUADRO RTX 4000 NV": 0.28,
                "RTX 3060 Ti NO": 0.34,
                "RTX 3090 NO": 0.59,
                "RTX 3090 IS": 0.59,
                "RTX 3080 NO": 0.50,
                "RTX 3080 IS": 0.50,
                "V100 IN": 1.07
                }


            # location 
            location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
                # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
                # "RTX 3090 (TY)": "Valencia, Spain",
                "RTX 3090 (UiS)": "Stavanger, Norway",
                "A100 80GB": "Illinois, USA", 
                "A100 40GB": "Illinois, USA", 
                "V100 IL": "Illinois, USA",
                "RTX A6000 IL": "Illinois, USA",
                "RTX A6000 NY": "New York, USA",
                "RTX A5000 IL": "Illinois, USA",
                "RTX A5000 NV": "Nevada, USA",
                "RTX A5000 NY": "New York, USA",
                "RTX A4000 NY": "New York, USA",
                "RTX A4000 IL": "Illinois, USA",
                "A40 IL": "Illinois, USA", 
                "A40 NY": "New York, USA", 
                "A40 NV": "Nevada, USA", 
                "QUADRO RTX 5000 IL": "Illinois, USA", 
                "QUADRO RTX 5000 NY": "New York, USA", 
                "QUADRO RTX 5000 NV": "Nevada, USA", 
                "QUADRO RTX 4000 IL": "Illinois, USA",
                "QUADRO RTX 4000 NY": "New York, USA",
                "QUADRO RTX 4000 NV": "Nevada, USA",
                "RTX 3060 Ti NO": "Stavanger, Norway",
                "RTX 3090 NO": "Stavanger, Norway",
                "RTX 3090 IS": "Iceland",
                "RTX 3080 NO": "Stavanger, Norway",
                "RTX 3080 IS": "Iceland",
                "V100 IN": "India"
                }

            # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
            SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                        "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                        # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                        "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                        "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                        "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                        "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                        "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                        }

            # print(trans_time)
            # lantency = No.of patches x 1/throughput_T4
            # throughput_T4 = 200   
            # time = []

            time = [n/throughput_T4 for n in datasetSize] 
            # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
            # transmission  = No.of patches * 100KB / speed(upload)    # Let 
            trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
                "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
                }


            # compute time
            T3090 = [round(i/2.23, 2) for i in time]
            compt_d = {"Tesla T4 (gpu-e)": time,
                # "Tesla T4 (gpu-f)": time,
                # "RTX 3090 (TY)": T3090,
                "RTX 3090 (UiS)": T3090,
                "A100 80GB": [round(i*1.55, 2) for i in T3090], 
                "A100 40GB": [round(i*1.55, 2) for i in T3090], 
                "V100 IL": [round(i*1.62, 2) for i in T3090],
                "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
                "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
                "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
                "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
                "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
                "A40 IL": [round(i*0.93, 2) for i in T3090], 
                "A40 NY": [round(i*0.93, 2) for i in T3090], 
                "A40 NV": [round(i*0.93, 2) for i in T3090], 
                "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
                "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
                "RTX 3090 NO": T3090,
                "RTX 3090 IS": T3090,
                "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
                "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
                "V100 IN": [round(i*1.62, 2) for i in T3090]
                }

            # tot_time = compt_time+trans_time
            computeDf = pd.DataFrame(data=compt_d)
            transDF = pd.DataFrame(data=trans_time)

            tottDf = computeDf+transDF
            costDf = tottDf * UP
            print(tottDf)


            
            time_matrix = tottDf.to_numpy()
            cost_matrix = costDf.to_numpy()
            print(costDf)
            

            n, m = time_matrix.shape  # n行m列
            print(n, m)

            # 定义最大总成本ε
            # max_total_cost = 100  # 这个值根据实际情况调整
            # max_total_cost = 120  # 这个值根据实际情况调整
            max_total_cost = budget  # 这个值根据实际情况调整

            # 创建问题实例
            prob = pulp.LpProblem("MinimizeMaxTimeUnderCostConstraint", pulp.LpMinimize)

            # 定义决策变量
            x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')
            # 定义一个辅助变量来表示最大完成时间
            max_time = pulp.LpVariable("max_time", lowBound=0)

            # 目标函数：最小化最大完成时间
            prob += max_time

            # 约束条件
            # 每行恰好选择一个元素
            for i in range(n):
                prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1

            # 每列最多选择一个元素
            for j in range(m):
                prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1

            # 总成本不超过ε
            prob += pulp.lpSum(x[(i, j)] * cost_matrix[i, j] for i in range(n) for j in range(m)) <= max_total_cost

            # 最大完成时间约束
            for i in range(n):
                for j in range(m):
                    prob += x[(i, j)] * time_matrix[i, j] <= max_time

            # 求解问题
            prob.solve()

            # 检查是否有最优解
            if pulp.LpStatus[prob.status] == "Optimal":
                solution = [j for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) > 0.5]
                total_cost = sum(cost_matrix[i, solution[i]] for i in range(n))
                max_time_value = pulp.value(max_time)
                assignment = [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
                print(f"Solution: {solution}, Total Cost: {total_cost}, Max Time: {max_time_value}, Assignment: {assignment}")
                solution_per_strategy = {'Total Cost': total_cost,
                                    'Max Time': max_time_value,
                                    'Solution': solution,
                                    'assignment': assignment }
                Even_solutions4strategies[(my_strategy, dataset_number)] = solution_per_strategy
            else:
                print("No optimal solution found.")
                continue 

            print(my_strategy) 
            
            # # 打印解决方案
            # for solution in solutions:
            #     print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
            # # print(solutions)

is_shuffled_True 4 [25.505, 25.505, 25.5, 25.5]
[5101, 5101, 5100, 5100]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  \
0        126.866153       54.422937  201.384365  201.384365  202.184365   
1        126.866153       54.422937  201.384365  201.384365  202.184365   
2        126.841282       54.404510  201.338362  201.338362  202.138362   
3        126.841282       54.404510  201.338362  201.338362  202.138362   

   RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  ...  \
0    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
1    193.954365    148.351421    197.614365    145.386731    152.011421  ...   
2    193.908362    148.314357    197.558362    145.340966    151.964357  ...   
3    193.908362    148.314357    197.558362    145.340966    151.964357  ...   

   QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  \
0          151.786731          210.884365          165.281421   
1          151.78673

In [19]:
print(Even_solutions4strategies)
print(len(Even_solutions4strategies.values()))
# print(Even_solutions4strategies.keys())
# print(Even_solutions4strategies.values())

{('is_shuffled_True', 4): {'Total Cost': 81.48572117126606, 'Max Time': 62.86451, 'Solution': [1, 22, 24, 21], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21)]}, ('is_shuffled_True', 5): {'Total Cost': 99.05702350257859, 'Max Time': 57.398337, 'Solution': [1, 21, 24, 23, 22], 'assignment': [(0, 1), (1, 21), (2, 24), (3, 23), (4, 22)]}, ('is_shuffled_True', 6): {'Total Cost': 82.5470036736782, 'Max Time': 84.560854, 'Solution': [1, 22, 23, 24, 21, 0], 'assignment': [(0, 1), (1, 22), (2, 23), (3, 24), (4, 21), (5, 0)]}, ('is_shuffled_True', 7): {'Total Cost': 91.7310915552998, 'Max Time': 72.473626, 'Solution': [21, 23, 25, 22, 24, 1, 0], 'assignment': [(0, 21), (1, 23), (2, 25), (3, 22), (4, 24), (5, 1), (6, 0)]}, ('is_shuffled_False', 4): {'Total Cost': 81.48572117126606, 'Max Time': 62.86451, 'Solution': [1, 22, 24, 21], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21)]}, ('is_shuffled_False', 5): {'Total Cost': 99.05702350257859, 'Max Time': 57.398337, 'Solution': [1, 21, 24, 23, 22

In [17]:
print(solutions4strategies)

{'saturation_largest_first': {'Total Cost': 81.49341761112282, 'Max Time': 62.847657, 'Solution': [21, 1, 22, 24], 'assignment': [(0, 21), (1, 1), (2, 22), (3, 24)]}, 'smallest_last': {'Total Cost': 79.21221733726564, 'Max Time': 60.115429, 'Solution': [24, 1, 22, 21, 0], 'assignment': [(0, 24), (1, 1), (2, 22), (3, 21), (4, 0)]}, 'connected_sequential': {'Total Cost': 86.17776927628933, 'Max Time': 58.671116, 'Solution': [1, 24, 22, 21, 18, 0], 'assignment': [(0, 1), (1, 24), (2, 22), (3, 21), (4, 18), (5, 0)]}, 'independent_set': {'Total Cost': 90.26339986885922, 'Max Time': 57.216804, 'Solution': [24, 22, 1, 21, 18, 8, 25], 'assignment': [(0, 24), (1, 22), (2, 1), (3, 21), (4, 18), (5, 8), (6, 25)]}, 'largest_first': {'Total Cost': 85.667821291027, 'Max Time': 44.986267, 'Solution': [22, 1, 24, 21, 25, 0, 23, 20], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 25), (5, 0), (6, 23), (7, 20)]}, 'random_sequential': {'Total Cost': 97.26354823929992, 'Max Time': 43.993731, 'Solut